In [ ]:
import os
!git clone https://github.com/ibrahim-elsawy/dstilPegasus
# !pip install -r /content/konafa/colab_req.txt
!pip install datasets
#!pip install torch==1.7.1
!pip install transformers==4.3.0rc1
!pip install sentencepiece 
!pip install catalyst==20.6
!pip install rouge-score

Cloning into 'dstilPegasus'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (344/344), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 344 (delta 223), reused 63 (delta 35), pack-reused 0
Receiving objects: 100% (344/344), 126.48 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (223/223), done.
     |████████████████████████████████| 225kB 7.5MB/s 
     |████████████████████████████████| 112kB 11.9MB/s 
     |████████████████████████████████| 245kB 14.5MB/s 
     |████████████████████████████████| 2.8MB 6.7MB/s 
     |████████████████████████████████| 3.2MB 49.1MB/s 
     |████████████████████████████████| 901kB 52.4MB/s 
     |████████████████████████████████| 1.2MB 8.4MB/s 
     |████████████████████████████████| 368kB 7.4MB/s 
     |████████████████████████████████| 174kB 31.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 122kB 34.2MB/s 
     |████████████████████████████████| 71kB 10

In [ ]:
os.chdir('/content/dstilPegasus')
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
from src.runners import DistilMLMRunnerFT
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
    SmoothingLossCallback,
    CrossentropylossCallback
)
import pandas as pd
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling
import numpy as np
from transformers.data.data_collator import default_data_collator
from torch import nn
from typing import Dict, List
from datasets import load_dataset
import datasets
from datasets import Dataset

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive")

Mounted at /content/drive


In [ ]:
train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
valid_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumvalidDataset_2")
teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda'))#

In [ ]:
train_dataset['decode_ids'][0]

AttributeError: ignored

In [ ]:
import datasets
import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline
students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },
}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))


def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(
            f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]


def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student


In [ ]:
student = create_student_with_configuration(teacher,
                                      e=16,
                                      d=4,
                                      copy_first_teacher_layers = False,
                                      save_path='./student').to('cuda')
model = torch.nn.ModuleDict({"student": student})
del teacher

In [ ]:
for param in student.model.shared.parameters():
  param.requires_grad = False
for param in student.model.encoder.parameters():
  param.requires_grad = False
#for param in student.model.encoder.embed_tokens.parameters():
#  param.requires_grad = False
#for param in student.model.encoder.embed_positions.parameters():
#  param.requires_grad = False
#for param in student.model.decoder.embed_tokens.parameters():
#  param.requires_grad = False
#for param in student.model.decoder.embed_positions.parameters():
#  param.requires_grad = False

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi


Sun May 23 01:35:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |   2456MiB / 15109MiB |     47%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
#valid_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
train_data = Dataset.from_dict(train_dataset[0:50000])
valid_data = Dataset.from_dict(valid_dataset[:5000])

train_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')
valid_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [ ]:
train_data['decode_ids'][0]

tensor([ 6320,   133,   243,   120,   114,  5318,   115,   109, 14126,  2947,
          117,  3793,   181,   780,   112,   129,  3231,   112,   462,   338,
          727,   107,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       device='cuda:0')

In [ ]:
train_dataloader = DataLoader(
    #train_dataset['train']
    train_data, batch_size=16
)
valid_dataloader = DataLoader(
    valid_data, batch_size=16
)
loaders = {"train": train_dataloader, "valid": valid_dataloader}
# del train_dataset
# del valid_dataset
#teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda')).to('cuda')
#student = torch.load('st_3dec_3enc.pt', map_location=torch.device('cuda')).to('cuda')
#model = torch.nn.ModuleDict({"teacher": teacher, "student": student})

In [ ]:
callbacks = {
    # "masked_lm_loss": MaskedLanguageModelCallback(),
    #"mse_loss": MSELossCallback(),
    "cross_ent_loss":CrossentropylossCallback(),
    #"cosine_loss": CosineLossCallback(),
    #"kl_div_loss": KLDivLossCallback(),
    #"label_smooth_loss": SmoothingLossCallback(),
    "loss": MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum",
        metrics={
            #"cosine_loss": 3.0,
            # "masked_lm_loss": 1.0,
            #"kl_div_loss": 0.8,
            "cross_ent_loss" : 1,
            #"mse_loss": 3.0,
            #"label_smooth_loss": .2,
        }
    ),
    "optimizer": dl.OptimizerCallback(),
    # "perplexity": PerplexityMetricCallbackDistillation()
}

In [ ]:
runner = DistilMLMRunnerFT(device=torch.device("cuda"))
runner.device = torch.device("cuda")
#optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
runner.train(
    model=model,
    optimizer=optimizer, 
    loaders=loaders,
    verbose=True,
    #check=True,
    callbacks=callbacks,
    num_epochs = 2,
    #check = True,
    # engine='cpu',
)

1/2 * Epoch (train):   0% 0/3125 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
train_data['input_ids']

RuntimeError: ignored